# Traitement calcul concentration


In [36]:
import pandas as pd
import numpy as np

## Chargement data

In [37]:
archives = pd.read_csv("data_propre/ARCHIVES_DATA_TABLE.csv", sep=";")
bat = pd.read_csv("data_propre/BATIMENT_MONUMENT_DATA_TABLE.csv", sep=",")
biblio = pd.read_csv("data_propre/BIBLIOTHEQUE_DATA_TABLE.csv", sep=";")
festivals = pd.read_csv("data_propre/FESTIVALS_DATA_TABLE.csv", sep=";")
cine=pd.read_csv("data_propre/CINEMA_DATA_TABLE.csv", sep=";")
musee=pd.read_csv("data_propre/MUSEE_DATA_TABLE.csv", sep=";")
theatre=pd.read_csv("data_propre/THEATRE_DATA_TABLE.csv", sep=";")

dept= pd.read_csv("data_propre/DEPARTEMENT_DATA_TABLE.csv", sep=";")
communes = pd.read_csv("data_propre/COMMUNE_DATA_TABLE.csv", sep=",")

## Uniformisation data

In [38]:
def transfo(df):
    df["CP"] = df["CP"].astype(str).str.zfill(5)
    df['COMMUNE'] = df['COMMUNE'].str.upper()
    df["COMMUNE"]=df["COMMUNE"].replace("\'", "-", regex=False)
    df["COMMUNE"]=df["COMMUNE"].replace(r'\bST\b', 'SAINT', regex=True)
    df["COMMUNE"]=df["COMMUNE"].replace(r'\bSTE\b', 'SAINT', regex=True)
    df["COMMUNE"]=df["COMMUNE"].replace(r'\bHT\b', 'HAUT', regex=True)
    df["COMMUNE"]=df["COMMUNE"].replace(r'\b&\b', 'ET', regex=True)
    return(df)

In [39]:
archives = transfo(archives)
bat = transfo(bat)
biblio =transfo(biblio)
festivals = transfo(festivals)
cine=transfo(cine)
musee=transfo(musee)
theatre=transfo(theatre)

arch_res = archives[['CP', 'NOM','Type', 'LATTI', 'LONGI', 'COMMUNE']]
bat_res = bat[['CP','NOM', 'TYPE', 'LATTI', 'LONGI', 'COMMUNE']]
biblio_res = biblio[['CP','NOM', 'SOUS_TYPE', 'LATTI', 'LONGI', 'COMMUNE']]
fest_res = festivals[['CP','NOM', 'SOUS_TYPE', 'LATTI', 'LONGI', 'COMMUNE']]
cine_res = cine[['CP','NOM', 'SOUS_TYPE', 'LATTI', 'LONGI', 'COMMUNE']]
musee_res = musee[['CP','NOM', 'SOUS_TYPE', 'LATTI', 'LONGI', 'COMMUNE']]
theatre_res = theatre[['CP','NOM', 'SOUS_TYPE', 'LATTI', 'LONGI', 'COMMUNE']]

## Calcul Concentration Departements

In [40]:
def extraire_numero_departement(code_postal):
    if code_postal.startswith(("97", "98")):  # Cas des départements à trois chiffres
        return code_postal[:3]
    return code_postal[:2]  # Cas général pour les deux premiers chiffres

In [41]:
dept["numdep"]=dept["numdep"].astype("str")
liste_dep=dept["numdep"].unique()
liste_dep = ["20" if element == "2A" or element == "2B" else element for element in liste_dep]
df = pd.DataFrame(liste_dep, columns=["DEP"])

In [ ]:
#concatenation des toutes les offres
tt_donnee = pd.concat([arch_res, bat_res, biblio_res, fest_res, cine_res, musee_res, theatre_res], axis=0)
tt_donnee["CP"] = tt_donnee["CP"].astype(str).str.zfill(5)
tt_donnee["DEP"] = tt_donnee["CP"].apply(extraire_numero_departement)

#calcul des evenements
evenements_par_departement = tt_donnee.groupby("DEP").size().reset_index(name="NombreEvenements")

#jointure
df = df.merge(evenements_par_departement, on="DEP", how="left")
df["Concentration"]=1000*df["NombreEvenements"]/df["Population"]

### Gestion des Nan

In [ ]:
df["Concentration"] = df["Concentration"].fillna(0)
nan_count = df.isna().sum()
print(nan_count)
df.to_csv("Concentration_dep.csv")

## Calcul Concentration Communes

In [44]:
#Agregation
evenements_par_commune = tt_donnee.groupby(["CP", "COMMUNE"]).size().reset_index(name="NombreEvenements")
communes = pd.read_csv("data_propre/COMMUNE_DATA_TABLE_1.csv", sep=",")
df=communes
df["CP"]=df["CP"].astype(str).str.zfill(5)

In [ ]:
#jointure
communes = df.rename(columns={'NOM': 'COMMUNE'})
df = communes.merge(evenements_par_commune, on=["CP", "COMMUNE"], how="left")

#calcul de la concentration
df["Concentration"]=1000*df["NombreEvenements"]/df["Population"]



### Gestion des Nan et valeurs infinies

In [ ]:
df["Concentration"] = df["Concentration"].fillna(0)
nan_count = df.isna().sum()
print(nan_count)
df["Concentration"] = df["Concentration"].replace([np.inf, -np.inf], 0)

### Visualisation des grandes valeurs

In [ ]:
test=df[df["Concentration"]>=100]
test

### telechargement donées

In [ ]:
df = df.rename(columns={ 'COMMUNE': 'NOM'})
df.to_csv("Concentration_communes.csv")

## Ajout des coordonnées des communes manquantes

### Chargement des données

In [ ]:
communes_rempli = pd.read_csv("data/correspondance-code-insee-code-postal.csv",sep=";")


#Compter les valeurs manquantes à ajouter
nan_count = communes.isna().sum()
print(nan_count)

communes["CP"]=communes["CP"].astype('str').apply(lambda x: x.zfill(5))


### Uniformisation data

In [ ]:
communes_rempli["Code Postal"]=communes_rempli["Code Postal"].astype('str').apply(lambda x: x.zfill(5))
communes_rempli["Commune"]=communes_rempli["Commune"].replace("\'", "-", regex=False)
communes_rempli["Commune"]=communes_rempli["Commune"].replace(r'\bST\b', 'SAINT', regex=True)
communes_rempli["Commune"]=communes_rempli["Commune"].replace(r'\bSTE\b', 'SAINT', regex=True)
communes_rempli["Commune"]=communes_rempli["Commune"].replace(r'\bHT\b', 'HAUT', regex=True)
communes_rempli["Commune"]=communes_rempli["Commune"].replace(r'\b&\b', 'ET', regex=True)
communes_rempli[['LATTI', 'LONGI']] = communes_rempli['geo_point_2d'].str.split(',', expand=True)
communes_rempli= communes_rempli.drop('geo_point_2d', axis=1)

### ajout des lattitude longitude manquante et geoshape

In [ ]:
# Assurez-vous que les colonnes clés ont le même format
communes["CP"] = communes["CP"].astype(str).apply(lambda x: x.zfill(5))
communes["NOM"] = communes["NOM"].str.strip()

communes_rempli["Code Postal"] = communes_rempli["Code Postal"].astype(str).apply(lambda x: x.zfill(5))
communes_rempli["Commune"] = communes_rempli["Commune"].str.strip()

# Merge sur les colonnes communes pour récupérer LATTI, LONGI, et geo_shape
merged = communes.merge(
    communes_rempli[["Code Postal", "Commune", "LATTI", "LONGI", "geo_shape"]],
    left_on=["CP", "NOM"],
    right_on=["Code Postal", "Commune"],
    how="left",
    suffixes=('', '_new')
)

# Compléter les valeurs manquantes
communes["LATTI"] = communes["LATTI"].fillna(merged["LATTI_new"])
communes["LONGI"] = communes["LONGI"].fillna(merged["LONGI_new"])
communes["geoshape"] = communes["geoshape"].fillna(merged["geo_shape"])

# Vérification finale des valeurs manquantes
nan_count = communes.isna().sum()
print("Valeurs manquantes après traitement :")
print(nan_count)




### Transformation en fichier csv

In [ ]:
print(df1[["LATTI", "LONGI", "geoshape"]].head())
df1.to_csv("test.csv")